# Jonasson Family Farm

This contains an example solution of a Linear Programming problem.  It also contains some code to help understand accessing values stored in arrays and dataframes.

## Prompt

This is based on Exercise 3.5-4 in Hillier, 11e.

> Fred Jonasson managed a family-owned farm.  To supplement several food products grown on the garm, Fred also raises pigs for market.  He now wishes to determine the quantities of the available types of feed (corn, tankage, and alfalfa) that should be given to each pig.
>
> Since pigs will any mix of these feed types, the objective is to determine which mix will meet certain nutritional requirements at a *minimum cost*.

| Nutritional Ingredient | Kg of Corn | Kg of Tankage | Kg of Alfalfa | Minimum Daily Requirement |
| -- | --- | --- | --- | --- |
| Carbohydrates | 90 | 20 | 40 | 200 |
| Protein | 30 | 80 | 60 | 180 |
| Vitamins | 10 | 20 | 60 | 150 |
| --- | --- | --- | --- | --- |
| Cost | \$10.50 | \$9.00 | \$7. 50 | |

# Solution

Jonasson farm seeks to find the quantities of feed which minimizes the costs, while meeting minimum nutritional requirements for the pigs.

In [1]:
# !pip install -q pulp
import numpy as np
import pandas as pd
import pulp as pl

In [2]:
feeds = ["Corn", "Tankage", "Alfalfa"]
ingredients = ["Carbs", "Protein", "Vitamins"]

# Each vector is Carbs, Protein, Vitamins, Costs 
corn = [90, 30, 10, 10.50]
tankage = [20, 80, 20, 9]
alfalfa = [40, 60, 60, 7.50]

# Build a dataframe with each row being a vector of properties of a feed type.
data = [corn, tankage, alfalfa]

df = pd.DataFrame(data = data,
                  index = feeds,
                  columns = ingredients + ["Cost"]
                 )

# Build a dictionary of minimum requirements for each ingredient.
minRequirements = [200, 180, 150]
minReqDict = dict(zip(ingredients, minRequirements))
# This can be created more directly with the following line:
# minReqDict = {"Carbs": 200, "Protein": 180, "Vitamins": 150}

The following table provides the amount of each ingredient and the cost per kilogram of ingredient.

In [3]:
df

,Carbs,Protein,Vitamins,Cost
Corn,90,30,10,10.5
Tankage,20,80,20,9.0
Alfalfa,40,60,60,7.5


The following table indicates the minimum requirements for each nutritional ingredient.

In [4]:
pd.DataFrame.from_dict(minReqDict, orient='index' , columns = ["Requirement"])

,Requirement
Carbs,200
Protein,180
Vitamins,150


The decision variables $X_f$ are the number of kilograms of each feed type to purchase.

In [5]:
lpVars = pl.LpVariable.dict("X", feeds, lowBound = 0)
lpVars

{'Corn': X_Corn, 'Tankage': X_Tankage, 'Alfalfa': X_Alfalfa}

We set up our model.

Let $F$ denote the set of feed types.  Let $I$ denote the set of nutritional ingredients.  $A = (a_{fi})$ denote the amount of nutritional ingredient $i \in I$ in feed $f \in F$. Let $C = (c_f)$, where $c_f$ is the cost of feed type $f \in F$.  $M = (m_i)$, where $m_i$ is the minimum daily requirement of ingredient $i \in I$.

We wish to minimize $$\sum_{f \in F} C_f X_f$$
subject to $$\sum_{f \in F} A_{fi} X_f \ge M_i, \quad i \in I$$


The matrix $A$ is specifically

In [6]:
df[ingredients].to_numpy()

array([[90, 30, 10],
       [20, 80, 20],
       [40, 60, 60]], dtype=int64)

We now build our model.

In [7]:
model = pl.LpProblem("Feed", pl.LpMinimize)

model += pl.lpSum( [ lpVars[feed] * df.at[feed, "Cost"] for feed in feeds])

for ingredient in ingredients:
    x = pl.lpSum([ lpVars[feed] * df.at[feed,ingredient] for feed in feeds])
    model += x >= minReqDict[ingredient], ingredient+"MinReq"
    
model

Feed:
MINIMIZE
7.5*X_Alfalfa + 10.5*X_Corn + 9.0*X_Tankage + 0.0
SUBJECT TO
CarbsMinReq: 40 X_Alfalfa + 90 X_Corn + 20 X_Tankage >= 200

ProteinMinReq: 60 X_Alfalfa + 30 X_Corn + 80 X_Tankage >= 180

VitaminsMinReq: 60 X_Alfalfa + 10 X_Corn + 20 X_Tankage >= 150

VARIABLES
X_Alfalfa Continuous
X_Corn Continuous
X_Tankage Continuous

In [8]:
status = model.solve()
print(pl.LpStatus[status])

Optimal


In [9]:
for v in model.variables():
  print(v.name, " = ", v.varValue)
print(f"Total Cost: ${pl.value(model.objective):.2f}")

X_Alfalfa  =  2.4285714
X_Corn  =  1.1428571
X_Tankage  =  0.0
Total Cost: $30.21


From this, we conclude that farmer Jonasson should purchase 2.4 kilograms of Alfalfa and 1.14 kilograms of corn for each pig for a minimal cost of $30.21.

## Notational Comments

We have a few different ways of accessing values within the arrays/dataframes/dictionaries.

In [10]:
# Value of a dictionary
minReqDict['Carbs']

200

In [11]:
# Value in a dataframe as a specific row and column
df.at['Corn','Carbs']

90

In [12]:
# Working with a specific variable
lpVars["Corn"]

X_Corn

In [13]:
# To get a row of a dataframe, the output is a "Series"
df.loc["Corn"]

Carbs       90.0
Protein     30.0
Vitamins    10.0
Cost        10.5
Name: Corn, dtype: float64

In [14]:
# To get the column of a dataframe
df["Carbs"]

Corn       90
Tankage    20
Alfalfa    40
Name: Carbs, dtype: int64

Note that type types of `ingredients` and `feeds` is a list.

In [15]:
print(ingredients)
print(feeds)

['Carbs', 'Protein', 'Vitamins']
['Corn', 'Tankage', 'Alfalfa']


You can easily loop through each item in a list.

In [16]:
for f in feeds:
    print("Feed Item:",f)

Feed Item: Corn
Feed Item: Tankage
Feed Item: Alfalfa


You can also create lists with the `for` loop inside them.  Here, we will create a new list which has the "Hello" affixed to the front of each feed item.

In [17]:
["Hello" + f for f in feeds]

['HelloCorn', 'HelloTankage', 'HelloAlfalfa']

This is useful for creating the products part of a dot product.

In [18]:
x = [ lpVars[feed] * df.at[feed,"Carbs"] for feed in feeds]
print("List of products:", x)
y = pl.lpSum(x)
print("Sum of productsion:", y)

List of products: [90*X_Corn + 0, 20*X_Tankage + 0, 40*X_Alfalfa + 0]
Sum of productsion: 40*X_Alfalfa + 90*X_Corn + 20*X_Tankage
